In [1]:
# torch imports
import torch
# Misc imports
import os 
import sys
import seaborn as sns
from pathlib import Path
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')
# Local imports
from ese.experiment.analysis.analysis_utils.submit_utils import get_ese_inference_configs
# Ionpy imports
from ionpy.util import Config
from ionpy.analysis import ResultsLoader

# Define some useful paths.
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
       '/storage'
))
# Set some defaults
rs = ResultsLoader()
sns.set_style("darkgrid")
torch.set_printoptions(linewidth=200)

# For using code without restarting.
%load_ext autoreload
%autoreload 2
# For using yaml configs.
%load_ext yamlmagic

In [2]:
%%yaml defaults_cfg 

experiment:
    exp_root: '?'
    inference_seed: 40
    hard_pred_threshold: 0.5 

log:
    root: '?'
    log_interval: 10 
    log_image_stats: True 
    log_pixel_stats: False 
    gether_inference_stats: True
    summary_compute_global_metrics: False 

dataloader:
    batch_size: '?'
    num_workers: '?' 
    pin_memory: True 

<IPython.core.display.Javascript object>

In [3]:
%%yaml calibration_cfg 

calibrator:
    _name: '?'

local_calibration:
    num_prob_bins: 15
    neighborhood_width: 3

global_calibration:
    num_classes: '?' 
    num_prob_bins: 15
    neighborhood_width: 3

<IPython.core.display.Javascript object>

In [4]:
%%yaml model_cfg 

# For standard datasets
#####################################
data:
    preload: False 

model:
    _type: "standard"
    pretrained_exp_root : None
    pretrained_select_metric: "val-dice_score"
    checkpoint: '?'
    pred_label: '?' 

<IPython.core.display.Javascript object>

## Gather Inference Options.

In [5]:
%%yaml experiment_cfg 

# name: '06_19_24_WMH_Hard'
name: '06_19_24_WMH_SVLS'

# base_models: "/storage/vbutoi/scratch/ESE/training/06_18_24_WMH_HardLabels"
base_models: "/storage/vbutoi/scratch/ESE/training/06_18_24_WMH_SVLS"

data:
    _class: "ese.data.datasets.WMH"
    annotator: "observer_o12"
    task: ("Amsterdam", "Singapore")
    split: ("train", "val", "cal")
    label_threshold: null 

<IPython.core.display.Javascript object>

In [6]:
# Get the configs for the different runs.
base_cfg = Config(defaults_cfg).update([calibration_cfg, model_cfg])
# Get the different experiment cfg yamls.
inf_cfgs = get_ese_inference_configs(
    exp_cfg=experiment_cfg, 
    base_cfg=base_cfg
)

WARNING! Calibrator not found, using default calibrator name: Uncalibrated.


In [7]:
len(inf_cfgs)

8

## Running Jobs

In [8]:
from ese.experiment.analysis.run_inference import get_cal_stats

/local/vbutoi/envs/UniverSegTF/lib/python3.9/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_outputs" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [9]:
from ese.experiment.experiment import run_ese_exp

###### Run individual jobs
run_ese_exp(
    config=inf_cfgs[0], 
    job_func=get_cal_stats,
    run_name='debug',
    # run_name=exp_name,
    show_examples=True,
    gpu='1',
) 

Set seed: 40


/storage/vbutoi/projects/ionpy/util/libcheck.py:49: UserWarning: Intel MKL extensions not available for NumPy
  warn("Intel MKL extensions not available for NumPy")
/storage/vbutoi/projects/ionpy/util/libcheck.py:57: UserWarning: Using slow Pillow instead of Pillow-SIMD
  warn("Using slow Pillow instead of Pillow-SIMD")


Running:

base_models: /storage/vbutoi/scratch/ESE/training/06_18_24_WMH_SVLS
calibrator:
  _class: Uncalibrated
  _name: Uncalibrated
data:
  annotator: observer_o12
  input_type: volume
  label_threshold: null
  num_slices: 1
  preload: false
  slicing: full
  split: ("train", "val", "cal")
  task: ("Amsterdam", "Singapore")
dataloader:
  batch_size: 2
  num_workers: 1
  pin_memory: true
dataset:
  _class: ese.experiment.datasets.WMH
  annotator: observer_o12
  augmentations: null
  axis: 0
  label_threshold: null
  num_slices: 1
  preload: false
  return_data_id: true
  slicing: full
  version: 0.2
experiment:
  dataset_name: WMH
  exp_name: 06_19_24_WMH_SVLS
  exp_root: /storage/vbutoi/scratch/ESE/inference/06_19_24_WMH_SVLS
  hard_pred_threshold: 0.5
  inference_seed: 40
  pretrained_seed: 44
  slices_per_fp: 5
global_cal_metrics:
- ECE:
    _fn: ese.experiment.metrics.calibration.ece_loss
    cal_type: toplabel
- Edge-ECE:
    _fn: ese.experiment.metrics.calibration.edge_ece_loss

TypeError: only integer tensors of a single element can be converted to an index

In [ ]:
# from ese.experiment.experiment import submit_ese_exps 

# #### Run Batch Jobs
# submit_ese_exps(
#     config_list=inf_cfgs,
#     job_func=get_cal_stats,
#     available_gpus=['0', '1', '2', '3']
# )